In [3]:
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
import nltk
from nltk import NaiveBayesClassifier
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [4]:
file = r".\Datasets\training_twitter_x_y_train.csv"
f2 = r".\Datasets\test_twitter_x_test.csv"

In [5]:
df = pd.read_csv(file)

y_train = df['airline_sentiment']
x_train = df.drop(['airline_sentiment'], axis = 1)

In [6]:
x_train.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [7]:
y_train.head()

0    negative
1    positive
2    positive
3    negative
4    negative
Name: airline_sentiment, dtype: object

In [8]:
x_train = x_train.drop(['tweet_id'], axis = 1)
x_train = x_train.drop(['airline_sentiment_gold'], axis = 1)
x_train = x_train.drop(['negativereason_gold'], axis = 1)

In [9]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10980 entries, 0 to 10979
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   airline         10980 non-null  object
 1   name            10980 non-null  object
 2   retweet_count   10980 non-null  int64 
 3   text            10980 non-null  object
 4   tweet_coord     776 non-null    object
 5   tweet_created   10980 non-null  object
 6   tweet_location  7430 non-null   object
 7   user_timezone   7403 non-null   object
dtypes: int64(1), object(7)
memory usage: 686.4+ KB


In [10]:
x_train.head()

,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,United,LocalKyle,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,Southwest,amccarthy19,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,United,J_Okayy,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [11]:
df2 = pd.read_csv(f2)

df2.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [12]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3660 entries, 0 to 3659
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   tweet_id                3660 non-null   int64 
 1   airline                 3660 non-null   object
 2   airline_sentiment_gold  9 non-null      object
 3   name                    3660 non-null   object
 4   negativereason_gold     8 non-null      object
 5   retweet_count           3660 non-null   int64 
 6   text                    3660 non-null   object
 7   tweet_coord             243 non-null    object
 8   tweet_created           3660 non-null   object
 9   tweet_location          2477 non-null   object
 10  user_timezone           2417 non-null   object
dtypes: int64(2), object(9)
memory usage: 314.7+ KB


In [13]:
x_test = df2.copy()

In [14]:
x_test = x_test.drop(['tweet_id'], axis = 1)
x_test = x_test.drop(['airline_sentiment_gold'], axis = 1)
x_test = x_test.drop(['negativereason_gold'], axis = 1)

In [15]:
x_test.head()

,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,American,zsalim03,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,American,sa_craig,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,Southwest,DanaChristos,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,US Airways,rossj987,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,American,tranpham18,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [16]:
def get_simple_pos(word):
    pos = pos_tag([word])[0][1]
    
    if pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('N'):
        return wordnet.NOUN
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
    
def clean_review(words):
    lemmatizer = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    punctuations = list(string.punctuation)
    stop.update(punctuations)
    output = []
    for word in words:
        if word.lower() not in stop:
            pos = pos_tag([word])
            clean_word = lemmatizer.lemmatize(word, pos = get_simple_pos(word))
            output.append(clean_word.lower())
            
    return output

def get_feature_dict(words):
    doc_dict = {}
    word_set = set(words)
    
    for w in features:
        doc_dict[w] = w in word_set
            
    return doc_dict

In [17]:
text = x_train['text']
text2 = x_test['text']

In [18]:
documents = []

for i in range(len(y_train)):
    documents.append((word_tokenize(text[i]),y_train[i]))

In [19]:
documents[0]

(['@',
  'SouthwestAir',
  'I',
  'am',
  'scheduled',
  'for',
  'the',
  'morning',
  ',',
  '2',
  'days',
  'after',
  'the',
  'fact',
  ',',
  'yes',
  '..',
  'not',
  'sure',
  'why',
  'my',
  'evening',
  'flight',
  'was',
  'the',
  'only',
  'one',
  'Cancelled',
  'Flightled'],
 'negative')

In [20]:
test_documents = []

for i in range(len(x_test)):
    test_documents.append(word_tokenize(text2[i]))

In [21]:
documents_cleaned = [(clean_review(doc),category) for doc, category in documents]

In [22]:
documents_cleaned[0]

(['southwestair',
  'schedule',
  'morning',
  '2',
  'day',
  'fact',
  'yes',
  '..',
  'sure',
  'even',
  'flight',
  'one',
  'cancelled',
  'flightled'],
 'negative')

In [25]:
test_documents_cleaned = [clean_review(doc) for doc in test_documents]

In [26]:
training_documents = documents_cleaned

In [27]:
testing_documents = test_documents_cleaned

testing_documents[0]

['americanair',
 'car',
 'gng',
 'dfw',
 'pulled',
 '1hr',
 'ago',
 'icy',
 'road',
 'on-hold',
 'aa',
 'since',
 '1hr',
 'ca',
 "n't",
 'reach',
 'arpt',
 'aa2450',
 'wat',
 '2']

In [28]:
all_words = []

for doc in training_documents:
    for w in doc[0]:
        all_words.append(w)

In [29]:
len(all_words) != len(set(all_words))  # repetition of words

True

In [30]:
freq = nltk.FreqDist(all_words)

In [31]:
freq.most_common(15)

[('flight', 3349),
 ('united', 3180),
 ('usairways', 2253),
 ('americanair', 2235),
 ('southwestair', 1841),
 ('jetblue', 1762),
 ('get', 1542),
 ("n't", 1535),
 ("'s", 1113),
 ('http', 906),
 ('hour', 863),
 ('thanks', 829),
 ('cancelled', 800),
 ('service', 742),
 ('help', 726)]

In [32]:
k = int(len(all_words) * 0.25)

features = [ word for word,frequency in freq.most_common(k)]

In [33]:
training_data = [(get_feature_dict(doc),category) for doc, category in training_documents]

In [34]:
training_data[0]

({'flight': True,
  'united': False,
  'usairways': False,
  'americanair': False,
  'southwestair': True,
  'jetblue': False,
  'get': False,
  "n't": False,
  "'s": False,
  'http': False,
  'hour': False,
  'thanks': False,
  'cancelled': True,
  'service': False,
  'help': False,
  'customer': False,
  'time': False,
  '...': False,
  'u': False,
  'call': False,
  'bag': False,
  'plane': False,
  'wait': False,
  'go': False,
  'amp': False,
  'need': False,
  'hold': False,
  'fly': False,
  "'m": False,
  'make': False,
  '2': True,
  'thank': False,
  'would': False,
  'still': False,
  'one': True,
  'gate': False,
  'day': True,
  'please': False,
  'back': False,
  'ca': False,
  'delayed': False,
  'try': False,
  'airline': False,
  'take': False,
  'virginamerica': False,
  'flightled': True,
  'say': False,
  'seat': False,
  "'ve": False,
  'delay': False,
  "''": False,
  '``': False,
  'like': False,
  'change': False,
  'phone': False,
  'bad': False,
  'agent': Fal

In [35]:
testing_data = [get_feature_dict(doc) for doc in testing_documents]

In [33]:
testing_data[0]

{'flight': False,
 'united': False,
 'usairways': False,
 'americanair': True,
 'southwestair': False,
 'jetblue': False,
 'get': False,
 "n't": True,
 "'s": False,
 'http': False,
 'hour': False,
 'thanks': False,
 'cancelled': False,
 'service': False,
 'help': False,
 'customer': False,
 'time': False,
 '...': False,
 'u': False,
 'call': False,
 'bag': False,
 'plane': False,
 'wait': False,
 'go': False,
 'amp': False,
 'need': False,
 'hold': False,
 'fly': False,
 "'m": False,
 'make': False,
 '2': True,
 'thank': False,
 'would': False,
 'still': False,
 'one': False,
 'gate': False,
 'day': False,
 'please': False,
 'back': False,
 'ca': True,
 'delayed': False,
 'try': False,
 'airline': False,
 'take': False,
 'virginamerica': False,
 'flightled': False,
 'say': False,
 'seat': False,
 "'ve": False,
 'delay': False,
 "''": False,
 '``': False,
 'like': False,
 'change': False,
 'phone': False,
 'bad': False,
 'agent': False,
 'late': False,
 'today': False,
 'know': False,
 

In [34]:
clf = NaiveBayesClassifier.train(training_data)

In [35]:
nltk.classify.accuracy(clf, training_data)

0.8877959927140255

In [36]:
y_pred = []

for i in testing_data:
    y_pred.append(clf.classify(i))

In [37]:
model2 = SklearnClassifier(SVC())
model2.train(training_data)

<SklearnClassifier(SVC())>

In [38]:
nltk.classify.accuracy(model2, training_data)

In [ ]:
y_pred_svm = []

for i in testing_data:
    y_pred_svm.append(model2.classify(i))

In [ ]:
model3 = SklearnClassifier(RandomForestClassifier())
model3.train(training_data)

In [ ]:
nltk.classify.accuracy(model3, training_data)

In [ ]:
y_pred_rf = []

for i in testing_data:
    y_pred_rf.append(model3.classify(i))

In [ ]:
np.savetxt("Project11_y_pred.csv", y_pred_rf,  delimiter = ",", fmt = "%s")